<a href="https://colab.research.google.com/github/ChiccoSy/BERT_Based_Multiclass_Text_Classification/blob/main/BERTopicExperience_No_ngram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic
!pip install nltk
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 20.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.4 MB/s eta 0:00:00
  Preparing metadata (s

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import re
import nltk
from bertopic import BERTopic
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
file_path = '/content/drive/MyDrive/Dissertation_UC/STUDENT_SURVEY_EXIT.csv'
data = pd.read_csv(file_path, encoding='ISO-8859-1')
data.columns = ['responses']

In [ ]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove numeric digits
    text = re.sub(r'\d+', '', text)

    return text
data['processed_responses'] = data['responses'].apply(preprocess_text)
stop_words = set(stopwords.words('english'))

def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

data['processed_responses'] = data['processed_responses'].apply(tokenize_and_remove_stopwords)
sample_preprocessed_responses = data['processed_responses'].sample(5)  # Display 5 random samples
for i, response in enumerate(sample_preprocessed_responses, 1):
    print(f"Sample {i}: {response}")

Sample 1: feel fortunate happy beneficiary able help family expenses even little aided siblings school needs paid school fees well consider tes blessing im eternally thanful first scholarship ever supporting tertiary education
Sample 2: exquisite beneficiary uaqte grateful
Sample 3: uaqte crucial part story college gave opportunity gain access quality education without much difficulty hope would continue exist help students hoping finished studies
Sample 4: fun great
Sample 5: beneficiary uaqte got chance pursue higher education without worry paying tuition fees effort gave equal access highquality education enabling get skills information needed thrive chosen career


In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 10  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=15,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-05 04:45:06,342 - BERTopic - Transformed documents to Embeddings
2023-10-05 04:45:18,597 - BERTopic - Reduced dimensionality
2023-10-05 04:45:18,874 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.7706180596001798
Coherence Score: 0.8461927569240604


In [ ]:
from collections import Counter

# Count how many documents are assigned to each topic
topic_sizes = Counter(topics)

# Sort topics by size/count in descending order
sorted_topics = sorted(topic_sizes.items(), key=lambda x: x[1], reverse=True)

# Calculate the total number of topics
total_topics = len(sorted_topics)

# Display the total number of topics
print(f"Total Number of Topics: {total_topics}\n")

# Display the sorted topics with their frequency, size/count, and all words
for topic_id, topic_size in sorted_topics:
    top_words = top_words_per_topic[topic_id]
    print(f"Topic {topic_id}: Size {topic_size} - Frequency {topic_size / len(documents):.2%}")
    print("All Words:", " ".join(word for sublist in top_words for word in sublist))
    print()


Total Number of Topics: 9

Topic 0: Size 1999 - Frequency 91.07%
All Words: less stress finances given enrolled demanding course parents also given hard time finance inspired better studies passing uaqte quite beneficial help struggling students attend school avail education provide filipinos equal opportunity quality tertiary education private public educational institutions although lot still improved cater needs student overall uaqte promising lucky enough one beneficiaries somehow helped lessen financial burden school beneficiary uaqte one factors considered choosing university go free tuition one beneficiary uaqte really opportunity college student grateful blessed assurance part always allowance every expenses school help parents benefited students need help uaqte big blessing mother considering longer need worry paying tuition fee helps focus studies compared wasnt implemented would working part time werent program things didnt cover still quite grateful helps family aside quali

In [ ]:
topic_model.get_topic(0)

[('education', 0.04620890024553821),
 ('uaqte', 0.043442491720441255),
 ('tuition', 0.03615919372725203),
 ('free', 0.03535195966747283),
 ('one', 0.030322809469914253),
 ('able', 0.028425444586701123),
 ('help', 0.027874113083358947),
 ('financial', 0.027539666186605912),
 ('college', 0.02700625219310161),
 ('students', 0.02633653359909192)]

In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 10  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=20,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-02 11:52:57,325 - BERTopic - Transformed documents to Embeddings
2023-10-02 11:53:10,281 - BERTopic - Reduced dimensionality
2023-10-02 11:53:10,430 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.7744150749061387
Coherence Score: 0.8144761929578446


In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 10  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=25,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-02 11:53:45,996 - BERTopic - Transformed documents to Embeddings
2023-10-02 11:53:59,033 - BERTopic - Reduced dimensionality
2023-10-02 11:53:59,181 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.8827328785541569
Coherence Score: 0.8399723551557777


In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 10  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=30,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-02 11:54:35,428 - BERTopic - Transformed documents to Embeddings
2023-10-02 11:55:04,796 - BERTopic - Reduced dimensionality
2023-10-02 11:55:04,951 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.9159696720570354
Coherence Score: 0.7204625402601121


In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 15  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=15,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-02 11:55:57,590 - BERTopic - Transformed documents to Embeddings
2023-10-02 11:56:11,056 - BERTopic - Reduced dimensionality
2023-10-02 11:56:11,212 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.7567061227149124
Coherence Score: 0.8339045466134019


In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 15  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=20,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-02 12:02:16,608 - BERTopic - Transformed documents to Embeddings
2023-10-02 12:02:30,919 - BERTopic - Reduced dimensionality
2023-10-02 12:02:31,072 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.7924050827629459
Coherence Score: 0.8253413331383391


In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 15  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=25,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-02 12:03:15,098 - BERTopic - Transformed documents to Embeddings
2023-10-02 12:03:28,631 - BERTopic - Reduced dimensionality
2023-10-02 12:03:28,783 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.8746697860920197
Coherence Score: 0.8294071740543284


In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 15  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=30,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-02 12:04:21,201 - BERTopic - Transformed documents to Embeddings
2023-10-02 12:04:40,149 - BERTopic - Reduced dimensionality
2023-10-02 12:04:40,305 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.908515176496484
Coherence Score: 0.7294918472791627


In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 20  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=15,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-05 01:22:20,118 - BERTopic - Transformed documents to Embeddings
2023-10-05 01:22:33,315 - BERTopic - Reduced dimensionality
2023-10-05 01:22:33,470 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.7617652343265862
Coherence Score: 0.8475451544650369


In [ ]:
from collections import Counter

# Count how many documents are assigned to each topic
topic_sizes = Counter(topics)

# Sort topics by size/count in descending order
sorted_topics = sorted(topic_sizes.items(), key=lambda x: x[1], reverse=True)

# Calculate the total number of topics
total_topics = len(sorted_topics)

# Display the total number of topics
print(f"Total Number of Topics: {total_topics}\n")

# Display the sorted topics with their frequency, size/count, and all words
for topic_id, topic_size in sorted_topics:
    top_words = top_words_per_topic[topic_id]
    print(f"Topic {topic_id}: Size {topic_size} - Frequency {topic_size / len(documents):.2%}")
    print("All Words:", " ".join(word for sublist in top_words for word in sublist))
    print()


Total Number of Topics: 9

Topic 0: Size 1999 - Frequency 91.07%
All Words: helps lessen struggles paying tuitions school related payments uaqte able benefit us students really need access quality education without thinking tuition greatly helped beloved parents especially single parent mom also beneficial trying times inflation greatly experienced experience ive gotten serious studying given chance become beneficiary uaqte thankful school year one hardest terms financial needs im case receive educational assistance helpful great experience one beneficiaries uaqte give lot helps student im really happy one beneficiaries uaqte benificiary uaqte thankful part experience definitely decreased load expenses school moreover able part made us save tons money family genuinely thankful allowing us partake got help scholarship like learning materials helped family tough financial situation times im grateful batch worry paying tuition fees rather focused medical expenses covid protocols school pr

In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 20  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=20,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-02 12:08:10,804 - BERTopic - Transformed documents to Embeddings
2023-10-02 12:08:25,693 - BERTopic - Reduced dimensionality
2023-10-02 12:08:25,859 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.7762770898617588
Coherence Score: 0.83439065821748


In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 20  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=25,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-02 12:09:17,228 - BERTopic - Transformed documents to Embeddings
2023-10-02 12:09:29,294 - BERTopic - Reduced dimensionality
2023-10-02 12:09:29,545 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.8697923104427754
Coherence Score: 0.8477970488588529


In [ ]:
# THIS INCLUDES BOTH SILHOUETTE AND COHERENCE SCORINGS
# Import the necessary libraries
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

# Define the number of topics
num_topics = 30  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=25,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Fit and transform the data for topic modeling
topics, probs = topic_model.fit_transform(documents)

# Determine the actual number of topics generated by the model
actual_num_topics = probs.shape[1]

# Use the actual number of topics for further processing
num_topics = actual_num_topics

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(probs, topics)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = (-probs[:, topic_id]).argsort()[:topic_model.top_n_words]
    top_words = [documents[i].split() for i in top_word_indices]
    top_words_per_topic.append(top_words)

# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-05 02:32:30,143 - BERTopic - Transformed documents to Embeddings
2023-10-05 02:32:42,001 - BERTopic - Reduced dimensionality
2023-10-05 02:32:42,155 - BERTopic - Clustered reduced embeddings


Silhouette Score: 0.8049629715189378
Coherence Score: 0.8509856203205581


In [ ]:
from collections import Counter

# Count how many documents are assigned to each topic
topic_sizes = Counter(topics)

# Sort topics by size/count in descending order
sorted_topics = sorted(topic_sizes.items(), key=lambda x: x[1], reverse=True)

# Calculate the total number of topics
total_topics = len(sorted_topics)

# Display the total number of topics
print(f"Total Number of Topics: {total_topics}\n")

# Display the sorted topics with their frequency, size/count, and all words
for topic_id, topic_size in sorted_topics:
    top_words = top_words_per_topic[topic_id]
    print(f"Topic {topic_id}: Size {topic_size} - Frequency {topic_size / len(documents):.2%}")
    print("All Words:", " ".join(word for sublist in top_words for word in sublist))
    print()


Total Number of Topics: 6

Topic 0: Size 2000 - Frequency 91.12%
All Words: beneficiary uaqte say tremendous help enabled pursue desired course without tuition fees additionally university offers highquality education thankful one beneficiaries universal access quality education really help study hard especially tertiary education given opportunity reach dream thru program know everyone privilege financially including im privilege program help aim aspire professional someday something surely owe program behind story dreamer benefit really helpful lessen family financial burden give extra motivation every student significant experience share aside decreased schooling expenses chance avail free education big university big blessing also advantage graduate graduating well known university great privilege quality education free education study university never get scholarship able free education without worrying expenses required tuition helps childrenstudent developflourish im grateful pa

In [ ]:
topic_model.visualize_topics()

In [ ]:
# c-TF-IDF implementation
# Import the necessary libraries
import numpy as np
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the number of topics
num_topics = 10  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=15,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
    n_gram_range=(1, 1)  # N-gram range (optional)
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Calculate C-TF-IDF scores first
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Fit the BERTopic model
topics, _ = topic_model.fit_transform(documents)

# Convert topics to a NumPy array and reshape it
topics_array = np.array(topics).reshape(-1, 1)

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(topics_array, topics_array)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = [
        i for i, prob in enumerate(topics) if prob == topic_id
    ]
    top_word_indices = sorted(top_word_indices, key=lambda i: -probs[i, topic_id])
    top_words = [documents[i].split() for i in top_word_indices[:topic_model.top_n_words]]
    top_words_per_topic.append(top_words)


# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-02 12:23:47,265 - BERTopic - Transformed documents to Embeddings
2023-10-02 12:24:00,400 - BERTopic - Reduced dimensionality
2023-10-02 12:24:00,556 - BERTopic - Clustered reduced embeddings


Silhouette Score: 1.0


IndexError: ignored

In [ ]:
# c-TF-IDF implementation
# Import the necessary libraries
import numpy as np
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the number of topics
num_topics = 10  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=20,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
    n_gram_range=(1, 1)  # N-gram range (optional)
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Calculate C-TF-IDF scores first
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Fit the BERTopic model
topics, _ = topic_model.fit_transform(documents)

# Convert topics to a NumPy array and reshape it
topics_array = np.array(topics).reshape(-1, 1)

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(topics_array, topics_array)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = [
        i for i, prob in enumerate(topics) if prob == topic_id
    ]
    top_word_indices = sorted(top_word_indices, key=lambda i: -probs[i, topic_id])
    top_words = [documents[i].split() for i in top_word_indices[:topic_model.top_n_words]]
    top_words_per_topic.append(top_words)


# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


Batches:   0%|          | 0/69 [00:00<?, ?it/s]

2023-10-02 12:25:21,417 - BERTopic - Transformed documents to Embeddings
2023-10-02 12:25:37,553 - BERTopic - Reduced dimensionality
2023-10-02 12:25:37,710 - BERTopic - Clustered reduced embeddings


Silhouette Score: 1.0


IndexError: ignored

In [ ]:
# c-TF-IDF implementation
# Import the necessary libraries
import numpy as np
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the number of topics
num_topics = 10  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=25,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
    n_gram_range=(1, 1)  # N-gram range (optional)
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Calculate C-TF-IDF scores first
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Fit the BERTopic model
topics, _ = topic_model.fit_transform(documents)

# Convert topics to a NumPy array and reshape it
topics_array = np.array(topics).reshape(-1, 1)

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(topics_array, topics_array)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = [
        i for i, prob in enumerate(topics) if prob == topic_id
    ]
    top_word_indices = sorted(top_word_indices, key=lambda i: -probs[i, topic_id])
    top_words = [documents[i].split() for i in top_word_indices[:topic_model.top_n_words]]
    top_words_per_topic.append(top_words)


# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")


In [ ]:
# c-TF-IDF implementation
# Import the necessary libraries
import numpy as np
from bertopic import BERTopic
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the number of topics
num_topics = 30  # Change this to your desired number of topics

# Initialize BERTopic with specified parameters
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=15,  # Minimum number of documents for a topic
    top_n_words=10,  # Number of top words to display for each topic
    n_gram_range=(1, 1)  # N-gram range (optional)
)

# Ensure all elements are strings
data['processed_responses'] = data['processed_responses'].apply(lambda x: str(x))

# Pass the data as a list of strings
documents = data['processed_responses'].tolist()

# Check and convert any non-string elements to strings
documents = [str(doc) if not isinstance(doc, str) else doc for doc in documents]

# Calculate C-TF-IDF scores first
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Fit the BERTopic model
topics, _ = topic_model.fit_transform(documents)

# Convert topics to a NumPy array and reshape it
topics_array = np.array(topics).reshape(-1, 1)

# Calculate the Silhouette Score
silhouette_avg = silhouette_score(topics_array, topics_array)
print(f"Silhouette Score: {silhouette_avg}")

# Extract top words for each topic
top_words_per_topic = []
for topic_id in range(num_topics):
    top_word_indices = [
        i for i, prob in enumerate(topics) if prob == topic_id
    ]
    top_word_indices = sorted(top_word_indices, key=lambda i: -probs[i, topic_id])
    top_words = [documents[i].split() for i in top_word_indices[:topic_model.top_n_words]]
    top_words_per_topic.append(top_words)


# Flatten the list of top words for each topic
flat_top_words = [word for sublist in top_words_per_topic for word in sublist]

# Convert BERTopic clusters into a format suitable for Gensim
docs = [doc.split() for doc in documents]
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Compute coherence scores using the flattened top words
coherence_model = CoherenceModel(
    texts=docs,  # Use the list of documents
    corpus=corpus,
    dictionary=dictionary,
    topics=flat_top_words,  # Provide flattened BERTopic top words
    coherence='c_v'  # You can choose other coherence measures as well
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")
